## 构建一个RAG应用
> RAG：Retrieval Augmented Generation

**官方文档：** https://python.langchain.com/docs/tutorials/rag/

**特别声明：** 本教程旨在跑通整个链路，模型效果暂不纳入考虑
### 整体方案
构建一个RAG简单应用主要包含两部分：文档索引和查询生成
1. 文档索引：构建知识文本索引，方便快速获取相似/相关文档。一般是离线的数据链路。
2. 获取并生成结果：实时获取相关文本并传给LLM，生成最终结果。

#### 技术方案
- Ollama + Chroma + LangChain：说明Ollama是本地运行大模型的框架；Chroma是向量数据库
- Embedding Model：nomic-embed-text（一个基于Sentence Transformers库的句子嵌入模型，专门用于特征提取和句子相似度计算）
- Chat Model：

LangChain相关库：langchain-ollama、langchain-chroma。安装指令：`pip install langchain-ollama langchain-chroma`

### 第一部分：构建文档索引
详见：03.langchain-vector-store

### 第二部分：查询生成
核心步骤：
1. 构建LLM模型实例
2. 构建Prompt Template
3. 构建Chain
4. 测试问题

LangChain提示词模板库：https://smith.langchain.com/hub

#### 第一步：初始化系统变量
主要包括2块：
- LANGSMITH_*: LangSmith相关的参数。
    - LANGSMITH_TRACING: 是否开启LangSmith追踪的开关
    - LANGSMITH_ENDPOINT: LangSmith平台访问URL
    - LANGSMITH_API_KEY: 访问LangSmith接口的API Key
    - LANGSMITH_PROJECT: 项目唯一标识
- DEEPSEEK_API_KEY: 访问DeepSeek大模型接口的API Key

In [5]:
import os
import getpass

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_PROJECT"] = "langchain-study-rag"
os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("请输入DeepSeek API KEY：")

请输入DeepSeek API KEY： ········


#### 第二步：构建LangGraph
##### 问题集合
1. HumanMessage、SystemMessage、ToolMessage这些消息是怎么组织并发给大模型的？
2. StateGraph的原理
3. LangGraph Platform的作用

##### 构建LangGraph需要定义三个东西
1. 应用的State
2. 应用的节点
3. 应用的控制流

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_deepseek import ChatDeepSeek
from langgraph.graph import START, END, StateGraph

from typing_extensions import List, TypedDict

prompt = ChatPromptTemplate([
    ("system", "你是一名资深供应链计划专家"),
    ("user", "相关领域知识:{knowledge} \n\n 问题:{question}")
])

chat_model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=1.0,
    max_retries=2
)

class State(TypedDict):
    documents: List[Document]
    question: str
    answer: str

# 从向量数据库获取知识文档集
def retrieve(state: State):
    documents = vector_store.similarity_search(state["question"])
    return {"documents": documents}

# 请求大模型
def generate(state: State):
    documents_content = "\n\n".join(document.page_content for document in state["documents"])
    messages = prompt.invoke({"question": state["question"], "knowledge": documents_content})
    response = chat_model.invoke(messages)
    return {"answer": response.content}

# 构建LangGraph
group_builder = StateGraph(State)
group_builder.add_node("retrieve", retrieve)
group_builder.add_node("generate", generate)
group_builder.add_edge(START, "retrieve")
group_builder.add_edge("retrieve", "generate")
group_builder.add_edge("generate", END)

rag_app = group_builder.compile()

In [8]:
result = rag_app.invoke({"question": "请问IBP是什么"})
print(result["answer"])

IBP（Integrated Business Planning，集成业务计划）是一种**战略性、跨职能的业务规划方法**，其本质不仅是规划工具，更是企业实现战略落地的管理平台。以下是其核心要点：

---

### **1. 定义与内涵**
- **战略整合**：将企业战略、财务目标与运营执行（销售、生产、供应链等）深度结合，打破部门孤岛。
- **滚动规划**：通过持续更新的动态计划（如18-24个月滚动周期），替代传统静态年度预算，提升敏捷性。
- **三位一体**：整合**数据**（统一视图）、**流程**（标准化协作）、**文化**（跨部门共识），驱动数字化转型。

---

### **2. 核心价值**
- **战略对齐**：确保执行层行动与企业长期战略一致。
- **动态响应**：快速应对市场需求波动、供应链中断等不确定性。
- **资源优化**：通过模拟分析（如"what-if"场景）平衡成本、库存与服务水平。

---

### **3. 关键实施流程**
1. **数据整合**：聚合财务、运营、市场数据，形成唯一可信来源。
2. **跨部门协同**：销售、生产、采购等职能共同参与决策（如S&OP会议）。
3. **动态迭代**：基于实时数据调整计划，并通过模拟预测不同决策的影响。

---

### **4. 与传统规划的区别**
| **维度**       | **IBP**                          | 传统规划                  |
|----------------|----------------------------------|-------------------------|
| **视角**       | 战略导向，长期与短期平衡         | 短期目标为主            |
| **协作方式**   | 跨部门实时协同                   | 部门独立规划            |
| **灵活性**     | 滚动更新，动态优化               | 固定年度计划            |
| **技术支撑**   | 依赖大数据、AI模拟               | 手工报表为主            |

---

### *